<h1><center>Práctica Aprendizaje: Airbnb.</center></h1>

<h1><center>Análisis Exploratorio de los datos</center></h1>

# Introducción

- Conjunto de datos: El conjunto de datos escogido es éste, extraído de Airbnb mediante técnicas de scraping. Dentro de las opciones recomiendo utilizar el extract (“Only the 14780 selected records”), ya que minimiza el tiempo de ejecución y evita problemas de memoria en equipos con menos prestaciones. airbnb-listing-extract.csv
- Se busca crear un evaluador automático de precios a partir de las carácterísticas de los alojamientos. 

# 0. Librerías

In [ ]:
# Paquetes y librerias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt  

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 1. Análisis exploratorio de los datos

## 1.1 Selección de variables

Vamos a comenzar con una exploración previa de los datos por si fuera necesario realizar cambios o transformaciones en los mismos.

En primer lugar, cargamos los datos y miramos la dimensión y estructura de los mismos.

In [ ]:
# Cargamos los datos
df = pd.read_csv("airbnb-listings-cleaned.csv")

# Creamos una copia para conservar la base de datos original
df_original = df.copy()

df.head(5)

In [ ]:
# Dimension de los datos
df.shape

Tenemos 4837 registros de 82 variables. Estas se explican en el <a href="https://github.com/sofiamacein/Airbnb_Proyecto#readme">README</a>

Vemos el tipo de las variables y si existen valores nulos.

In [ ]:
# Tipo de las variables y valores nulos
df.info()

**Selección de variables**

Para el estudio cualitativo del host, consideramos todas las variables referidas al mismo. 

- host_id
- host_name  
- host_since      
- host_location
- host_response_time
- host_response_rate      
- host_is_superhost
- host_neighbourhood
- host_listings_count
- host_total_listings_count      
- host_has_profile_pic
- host_identity_verified

Seleccionamos las siguientes variables para el modelo:

- latitude
- longitude
- room_type
- accommodates
- bathrooms
- bedrooms
- beds
- price
- security_deposit
- cleaning_fee
- guests_inclueded
- extra_people
- minimum_nights
- maximum_nights
- number_of_reviews
- review_scores_accuracy                  
- review_scores_cleanliness                   
- review_scores_checkin                  
- review_scores_communication        
- review_scores_location          

En primer lugar, creamos el dataframe para el estudio cualitativo del host

In [ ]:
# Comprobamos que la variables 'id' conforme un indice valido
print("Valores nulos id: " + str(df['id'].isnull().sum()))
print("Valores duplicados id: " + str(df['id'].duplicated().sum()))

In [ ]:
# Creamos un dataframe para el estudio cualitativo del host
col_host = [0,2,3,4,5,6,7,9,10,11,12,13,14]
df_host = df.iloc[:,col_host]

# Cambiamos el indice
df_host = df_host.set_index('id')

df_host.head(1)

Por otro lado, creamos el dataframe con el que se trabajará para la construcción de los modelos

In [ ]:
# Dataframe para el estudio
col_tasa = [0,26,27,30,31,32,33,34,37,40,41,42,43,44,45,59,64,65,66,67,68]
df = df.iloc[:,col_tasa]

# Cambiamos el indice
df = df.set_index('id')
df_original = df_original.set_index('id')

df.head(1)

## 1.2 Transformación de variables

Realizamos algunas transformaciones sobre las variables

### 1.2.1 Tranformaciones en el dataframe del host

En primer lugar, a partir de la variables _host_since_, creamos una variable que denominamos _host_for_ cuántos años lleva siendo host.

In [ ]:
# Nos quedamos solo con el año de host_since 
df_host['host_since'] = pd.to_datetime(df_host['host_since'], errors='coerce')
df_host['host_since'] = df_host['host_since'].dt.year
df_host['host_since'] = 2019 - df_host['host_since'].values
df_host = df_host.rename(columns={'host_since': 'host_for'})

Por otro lado, binarizamos las variables que toman valores *t* y *f*

In [ ]:
# Cambiamos variables dicotomicas a binario 1:t y 0:f
df_host['host_is_superhost'] = df_host['host_is_superhost'].map({"t":1, "f":0})
df_host['host_has_profile_pic'] = df_host['host_has_profile_pic'].map({"t":1, "f":0})
df_host['host_identity_verified'] = df_host['host_identity_verified'].map({"t":1, "f":0})

Por último, convertimos los valores de la variable _host_response_time_ a valores entre $0$ y $1$.

In [ ]:
# Cambiamos host_response_time a tasa
df_host['host_response_rate'] = df_host['host_response_rate'].str.replace('%','').astype('float64')
df_host['host_response_rate'] = df_host['host_response_rate']/100

Obtenemos el siguiente dataframe

In [ ]:
df_host.head(2)

### 1.2.2 Transformaciones en el dataframe de estudio

Continuamos con el dataframe para el estudio.

En primer lugar, como la variable _room_type_ toma 3 valores, *Entire homre/apt*, *Private room* y *Shared room*, obtenemos variables dummies para estas.

In [ ]:
# Cambiamos la variable room_type a dummies segun las 3 categorias
df['room_type'].value_counts()
df_rooms = pd.get_dummies(df[['room_type']], prefix = "", prefix_sep = "")
df = pd.concat([df,df_rooms], axis=1)
df.drop(['room_type'], axis = 1, inplace = True)

Procedemos a cambiar la variable _extra_people_ a una variable de tipo numérico (float).

In [ ]:
# Cambiamos extra_people a dtype=float64
df['extra_people'] = df['extra_people'].str.replace('$','').astype('float64')

Al observar el dataframe, parece que hay entradas donde la variable _accommodates_ toma valores inferiores a la variable _beds_. Si esto es así, habría ofertas donde hay más camas disposibles que gente admitida. En ese caso, decidimos cambiar la variable _accommodates_ al número mínimo de personas necesarias para cubirir todas las camas; es decir,

_accomodates_ = _beds_ - _guests_included_

In [ ]:
# Comprobar que existen ofertas donde hay mas camas que gente admitida
df[df['beds']>df['accommodates']+df['guests_included']][['accommodates','beds','guests_included']]

In [ ]:
# Reemplazamos 'accommodates' por el numero minimo de personas necesarias para cubrir las camas
df['accommodates'] = np.where(df['beds'] > df['accommodates']+df['guests_included'], df['beds']-df['guests_included'], df['accommodates'])

Por último, estudiamos cuántos casos existen donde hay más invitados que personas que pueden alojarse y el tipo de alojamientos donde esto sucede.

In [ ]:
# Vemos tipo de alojamiento cuando hay mas invitados que personas que pueden alojarse
indices = df[df['guests_included']>df['accommodates']].index
df_original.loc[indices,:]['property_type'].value_counts()

En la mayoría de casos se trata de alojamientos de grandes dimensiones. Más adelante se realizará un estudio más exhaustivo.

## 1.3  Análisis descriptivo

**Estudio cualitativo del host**

Podemos estudiar:
- ¿Existen variables influyentes que determinen si un host es superhost?
- Mapa de localización de los host

**Estudio de las variables para los modelos de machine learning**

Vemos algunos estadísticos de los datos y estudiamos la distribución de las variables mediante los histogramas.

In [ ]:
# Estadisticos descriptivos
df.drop(columns=['latitude', 'longitude']).iloc[:,0:12].describe()

In [ ]:
# Estadisticos descriptivos
df.drop(columns=['latitude', 'longitude']).iloc[:,12:].describe()

In [ ]:
# Histograma de las variables
df.drop(columns=['latitude', 'longitude']).hist(figsize=(20,15))
plt.show()

Podemos observar una gran asimetría por la derecha en las variables. Por tanto, tras realizar la imputación de missings y el estudio de los outliers, se estudiará el coeficiente de asimetría realizando las trasnformaciones necesarias.

## 1.3 Imputación de missings

In [ ]:
# Valores missing
df.isnull().sum()

### 1.3.1 Variables 'bedrooms' y 'beds'

Para imputar los missing de las variables *bedrooms* y *beds* empleamos un modelo de regresión lineal donde las variables regresoras son *accommodates* y *bathrooms*.

In [ ]:
# Librerias
from sklearn.linear_model import LinearRegression

En primer lugar, imputamos la variable **_bedrooms_**

In [ ]:
# Separamos las variables
target = 'bedrooms'
regresoras = ['accommodates', 'bathrooms']

# Creamos un dataframe con las variables seleccionadas
df_imp = df[[target] + regresoras]

# Separamos los datos en dos dataframe, uno con los valores missing y otro sin valores missing
df_missing = df_imp[df_imp.isnull().any(axis=1)]
df_no_missing = df_imp.dropna()

# Ajustamos el modelo a los datos sin missing
reg = LinearRegression()
reg.fit(df_no_missing[regresoras], df_no_missing[target])

# Realizamos la imputacion de los valores missing y almacenamos los valores obtenidos en el dataframe 
pred = reg.predict(df_missing[regresoras])
df.loc[df[target].isnull(), target]= pred

Imputamos los missing en la variable **_beds_**

In [ ]:
# Separamos las variables
target = 'beds'
regresoras = ['accommodates', 'bathrooms']

# Creamos un dataframe con las variables seleccionadas
df_imp = df[[target] + regresoras]

# Separamos los datos en dos dataframe, uno con los valores missing y otro sin valores missing
df_missing = df_imp[df_imp.isnull().any(axis=1)]
df_no_missing = df_imp.dropna()

# Ajustamos el modelo a los datos sin missing
reg = LinearRegression()
reg.fit(df_no_missing[regresoras], df_no_missing[target])

# Realizamos la imputacion de los valores missing y almacenamos los valores obtenidos en el dataframe 
pred = reg.predict(df_missing[regresoras])
df.loc[df[target].isnull(), target]= pred

### 1.3.2 Variables 'security_deposit' y 'cleaning_fee '

Consideramos que los valores NaN son $0$, es decir, indican que no se paga depósito ni tasas de limpieza. 

In [ ]:
# Reemplazamos los valores missing por 0
df['security_deposit'] = df['security_deposit'].replace(np.nan, 0)
df['cleaning_fee'] = df['cleaning_fee'].replace(np.nan, 0)

### 1.3.3 Variables tipo 'review_scores_'

Vemos el dataframe donde al menos una de las variables correspondientes con reseñas tiene un valor missing

In [ ]:
# Dataframe reseñas
review_score = ['review_scores_accuracy','review_scores_cleanliness', 'review_scores_checkin',
                  'review_scores_communication','review_scores_location']

df[df[review_score].isna().any(axis=1)][review_score]

Así, decidimos eliminar las observaciones

In [ ]:
# Eliminamos las observaciones
df = df.dropna(subset=review_score)

## 1.4 Outliers

En primer lugar, mostramos los boxplot de cada una de las variables.

In [ ]:
# Numero de filas y columnas necesarias para dibujar todos los boxplots
n_rows = 6
n_cols = 3
columnas = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee','guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'review_scores_accuracy','review_scores_cleanliness', 'review_scores_checkin','review_scores_communication', 'review_scores_location']

# Matriz de subgraficos
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(10, 8))

# Boxplot de cada variable
for i, columna in enumerate(columnas):
    ax = axs[i // n_cols][i % n_cols]
    ax.boxplot(df[columna],vert=False,patch_artist=True) # boxplot de la variable
    ax.set_title(columna)

# Ajuste de espacios entre subgraficos
fig.tight_layout()

# Mostrar la figura
plt.show()

Estudiamos los outliers en conjunto con aquellas variables con las que podrían tener más relación. La relación entre las variables podemos estudiarla mediante la matriz de correlaciones

In [ ]:
# Correlaciones
plt.figure(figsize=(20,10))
correlaciones= df.corr()
sns.heatmap(correlaciones,cmap="GnBu",annot=True) 
correlaciones

Dividimos las variables en distintos grupos en función de su correlación para analizarlas conjuntamente.

### 1.4.1 Variables relacionadas con la disposición del alojamiento

**Accommodate, bathrooms,  bedrooms, beds**

Primero vamos a estudiar los outliers en las variables relativas a la disposicion del alojamiento

In [ ]:
# Boxplot de las habitaciones
fig = plt.figure(figsize = (5,5))

plt.boxplot(df[['accommodates', 'bathrooms','bedrooms', 'beds']])

plt.title('Outliers')
plt.xticks([1,2,3,4], ['Accommodates', 'Bathrooms', 'Bedrooms', 'Beds'])

plt.show()

Estudiamos los outliers en conjunto. Para ello empleamos la librería *PyOD* que posee estrategias para detectar outliers. En este caso, empleamos el algoritmo de KNN

In [ ]:
# !pip install pyod

In [ ]:
# Librerias
from pyod.models.knn import KNN

In [ ]:
# Deteccion de outliers
A = df[['accommodates','bathrooms','bedrooms', 'beds', 'Entire home/apt', 'Private room']]
clf = KNN(contamination=0.01)
clf.fit(A)
# Outliers
y_pred = clf.predict(A)
outliers = A[y_pred == 1]
# Vemos el tipo de alojamiento
indices = outliers.index
outliers['property_type'] = df_original.loc[indices,:]['property_type']
outliers

Podríamos considerar outliers los que se trata de un apartamento dado que, por lo general, el número de posibles huéspedes es muy alto y las dimensiones de un apartamento no suelen ser tan grandes.

In [ ]:
# Dataframe con los outliers correspondientes a apartamentos
outliers[outliers['property_type']=='Apartment']

Podemos observar que parece que se trata de datos mal registrados.

- En el primer caso, parece que se trata de un apartamente con 10 habitaciones y (posiblemente 1 cama por habitación).
- En el segundo caso, parece que se trata de una habitación privada en un apartamento.
- En el tercer caso

Decidimos eliminar las observaciones.

In [ ]:
# Eliminamos las observaciones correspondientes con apartamentos
indices_apt = outliers[outliers['property_type']=='Apartment'].index
df.drop(indices_apt, inplace = True)

### 1.4.2 Variables relacionadas con tasas extra

**Security deposit, cleaning fee, guests included, extra people**

**Security deposit**

En primer lugar, vemos cómo se distribuye la variable y sus posibles outliers

In [ ]:
# Distribucion y boxplot de security deposit
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(df['security_deposit'], shade=True, color='blue',ax=axes[0])
sns.boxplot(df['security_deposit'], color='blue', ax=axes[1])

plt.show()

In [ ]:
# Estudiamos los cuantiles de la variable
df['security_deposit'].quantile([0.25,0.5, 0.75, 0.95, 0.99])

Las fianzas altas suelen estar relacionadas con largas estancias o con lugares de grandes dimensiones. Denominaremos fianza alta a aquellas por encima del percentil 0,95.

In [ ]:
# Dataframe con fianzas por encima del percentil 95
q95 = df['security_deposit'].quantile(0.95) 
df_fianza = df[df['security_deposit']>=q95][['minimum_nights','maximum_nights', 
                                             'price','security_deposit','Entire home/apt', 'Private room', 'Shared room']]
indices = df_fianza.index # indices 
df_fianza['property_type'] = df_original.loc[indices]['property_type'] # tipo de alojamiento

df_fianza

Podríamos considerar outliers aquellos que no sean larga estancia y/o no correspondan con un alojamiento de grandes dimensiones.

In [ ]:
# Tipos de alojamiento
df_fianza['property_type'].value_counts()

In [ ]:
# No son largas estancias si es menos de 4 meses? y tipo alojamiento pequeño (apartment, loft y other)?
df_fianza[(df_fianza['maximum_nights']<(30*4)) & 
          ((df_fianza['property_type']=='Apartment') | (df_fianza['property_type']=='Loft') | (df_fianza['property_type']=='Other'))]

Este dataframe muestra aquellas observaciones que no corresponden a largas estancias (menos de 4 meses) y además el alojamiento es "pequeño" (apartamento, loft y otros). Consideramos que son outliers y los eliminamos.

In [ ]:
# Eliminamos las observaciones 
indices_est = df_fianza[(df_fianza['maximum_nights']<(30*4)) & 
                        ((df_fianza['property_type']=='Apartment') | (df_fianza['property_type']=='Loft') | (df_fianza['property_type']=='Other'))].index
df.drop(indices_est, inplace = True)

**Cleaning fee**

En primer lugar, vemos la distribución de _cleaning_fee_ y sus posibles valores atípicos

In [ ]:
# Distribucion y boxplot de cleaning fee
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(df['cleaning_fee'], shade=True, color='blue',ax=axes[0])
sns.boxplot(df['cleaning_fee'], color='blue', ax=axes[1])

plt.show()

En el siguiente gráfico, representamos las tasas de limpieza para los distintos tipos de alojamiento.

In [ ]:
# Crear la figura y los ejes
fig, axs = plt.subplots()

# Tramar el gráfico de dispersión con diferentes categorías
axs.scatter(df[df['Entire home/apt'] == 1]['cleaning_fee'], [1] * len(df[df['Entire home/apt'] == 1]), label='Entire home/apt')
axs.scatter(df[df['Private room'] == 1]['cleaning_fee'], [2] * len(df[df['Private room'] == 1]), label='Private room')
axs.scatter(df[df['Shared room'] == 1]['cleaning_fee'], [3] * len(df[df['Shared room'] == 1]), label='Shared room')

# Añadir etiquetas a los ejes
axs.set_xlabel('Cleaning Fee')
axs.set_yticks([1, 2, 3])
axs.set_yticklabels(['Entire home/apt', 'Private room', 'Shared room'])

# Mostrar la leyenda
axs.legend()

# Mostrar el gráfico
plt.show()

Estudiamos la relación entre 'cleaning_fee' y 'review_scores_cleanliness'.

In [ ]:
# Relacion cleaning_fee y review_score_cleanliness
plt.scatter(df['cleaning_fee'],df['review_scores_cleanliness'])
plt.plot( [0, 500],  [0, 10] , color='red')

plt.xlabel('Cleaning Fee')
plt.ylabel('Review Scores Cleanliness')

plt.show()

Consideramos que todos los datos que están por debajo de la línea son outliers. Decidimos eliminarlos.

In [ ]:
# Eliminamos las observaciones
indices = df[(df['cleaning_fee']>100)&(df['review_scores_cleanliness']<3)].index
df.drop(indices, inplace=True)

**Guests included**

Podemos estudiar esta variable en conjunto de con número de huéspedes y el número de camas/habitaciones

In [ ]:
# Distribucion y boxplot de guests_included
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(df['guests_included'], bins=30, alpha=0.5, color='blue', ax=axes[0])
sns.boxplot(df['guests_included'], color='blue', ax=axes[1])

plt.show()

Consideramos outliers aquellos donde los invitados son más del doble de los posibles huéspedes.

In [ ]:
# Dataframe con posibles valores atipicos de guests_included
outliers_guests = df[df['guests_included']>2*df['accommodates']][['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
                                               'guests_included', 'Entire home/apt', 'Private room']]
outliers_guests

Decidimos eliminar las observaciones.

In [ ]:
# Eliminamos las observaciones
df.drop(outliers_guests.index, inplace=True)

**Extra people**

Podemos estudiar esta variable en conjunto con el tipo de alojamiento y 'guests_included'. En primer lugar, observamos su distribución y los posibles valores atípicos.

In [ ]:
# Distribucion y boxplot de extra_people
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(df['extra_people'], shade=True, color='blue',ax=axes[0])
sns.boxplot(df['extra_people'], color='blue', ax=axes[1])

plt.show()

In [ ]:
# Estudiamos los cuantiles de la variable
df['extra_people'].quantile([0.25,0.5, 0.75, 0.95, 0.99])

Consideramos posibles outliers aquellos por encima del percentil 0,99

In [ ]:
# Dataframe con posibles outliers extra_people
q99 = df['extra_people'].quantile(0.99)
outliers_extra = df[df['extra_people']>q99][['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'guests_included', 'extra_people', 
                            'Entire home/apt', 'Private room', 'Shared room']]
outliers_extra

No consideramos que ninguna de las observaciones sean outliers.

### 1.4.3 Variables relacionadas las reseñas

Las reseñas son muy subjetivas y pueden presentar mucha variabilidad. Por ejemplo, puede darse el caso de un alojamiento situado en una muy buena ubicación pero con pésimas condiciones de limpieza. 

**Variables de tipo review_score_**

Calculamos la puntuación media según el tipo de habitación

In [ ]:
reviews = df[['number_of_reviews','review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
              'review_scores_communication', 'review_scores_location']]

indices_reviews = reviews.index
reviews['room_type'] = df_original.loc[indices_reviews,:]['room_type']

# Valoraciones media segun el tipo de habitacion    
df_media = reviews.groupby(['room_type']).mean()
df_media.drop('number_of_reviews', axis=1, inplace=True)

In [ ]:
# Grafico puntuaciones medias
plt.figure(figsize=(12, 5))

df_media.plot(kind='bar')

plt.xlabel('Tipo de habitacion')
plt.ylabel('Puntuaciones medias')
plt.title('Puntuacion media por tipo de habitacion')
plt.legend(loc='best') # buscar mejor ubicacion

plt.show()

**idea**: Puntuación media por zonas

### 1.4.4 Variables relacionadas con las noches

**minimum**

En primer lugar, sacamos los estadísticos de esta variable

In [ ]:
# Estadisticos descriptivos
df['minimum_nights'].describe()

También podemos ver que existen algunos casos extraños donde la duración de la estancia es muy particular 

In [ ]:
# Intervalos de un dia entre min y max
df[df['maximum_nights']-df['minimum_nights']<2][['minimum_nights', 'maximum_nights']]

Decidimos borrar estos últimos 

In [ ]:
outliers_min = df[df['maximum_nights']-df['minimum_nights']<2][['minimum_nights', 'maximum_nights']]
indices_min = [1191143, 7787575, 11949456, 12434489, 13771205, 15819364, 15840160,
17438123, 18638835, 19101912, 25011581, 25750741, 25887824, 26424386,
28765752]

# Eliminamos las observaciones
df.drop(indices_min, inplace = True)

Procedemos a estudiar el resto de los casos. 

Observamos la distribución del número mínimo de noches

In [ ]:
# Distribucion de minimum_nights
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(df['minimum_nights'], shade=True, color='blue', ax=axes[0])
sns.histplot(df['minimum_nights'], color='blue', bins = 30, ax=axes[1])

plt.show()

Hay dos picos, uno para alquileres por noche (min 1 noche) y otro para alquieres por meses (min 30 noche). 

Se considera que pueden existir alquileres que exijan ciertos meses de permanencia. Por tanto, no se considera outlier ninguno de los anteriores casos

**maximum**

Estudiamos nuevamente los estadísticos de la variable

In [ ]:
# Estadisticos descriptivos
df['maximum_nights'].describe()

Se puede observar que hay muchas observaciones concentradas en el valor 1125 (posiblemente sea un valor por defecto).
Por tanto, imputamos este valor como cota máxima del número máximo de noches.

In [ ]:
# Imputacion de cota maxima
df.loc[df['maximum_nights'] > 1125, 'maximum_nights'] = 1125

Procedemos a estudiar la distribución del número máximo de noches.

In [ ]:
# Distribucion de maximum_nights
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(df['maximum_nights'], shade=True, color='blue', ax=axes[0])
sns.histplot(df['maximum_nights'], color='blue', bins = 30, ax=axes[1])

plt.show()

Nuevamente vemos como existen 2 picos. El primero que hace referencia a estancias de corta duración y otro que hace referencia estancias de larga duracion.

## 1. 5 Transformaciones 

Como habíamos comentado, las variables presentan asimetría, lo cual no es favorable para los modelos de regresión. Podemos estudiar esto mediante el coeficiente de asimetria de Fisher, que mida la centralidad de los datos. Si el coeficiente es 0, los datos están centrados entrorno a la media, si es postivo presentan asimetría por la derecha, y si es negativo asimetría por la izquierda.

In [ ]:
# Simetria
df.skew(axis=0)

Así, aplicamos transformaciones logarítmicas a las variables continuas que no presentan simetría con sesgo positivo.

In [ ]:
df_log = df.copy()
cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'security_deposit','cleaning_fee', 'guests_included', 
        'extra_people','number_of_reviews']

In [ ]:
# Transfomacion
cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 
        'security_deposit','cleaning_fee', 'guests_included', 
        'extra_people','number_of_reviews']

for col in cols:
    df_log[col] = df_log[col].astype('float64').replace(0.0, 0.01)
    df_log[col] = np.log(df_log[col])

## 1.6 Análisis de correlaciones

Para finalizar, estudiamos las correlaciones entre las variables. Así, podemos decidir qué variables usar para el modelo y estudiar posibles relaciones existentes entre ellas.

In [ ]:
plt.figure(figsize=(20,10))
correlaciones= df_log.corr()
sns.heatmap(correlaciones,cmap="GnBu",annot=True) 
correlaciones

Así, seleccionamos las siguientes variables para el modelo:

Las variables de modelización:
* latitude
* longitude
* bathrooms
* beds
* security_deposit
* cleaning_fee
* extra_people
* number_of_reviews
* review_scores_accuracy
* Entire home/apt
* Private room
* Shared room

Y la varibale de etiqueta:
* price

Creamos una nueva variable _review_ de la siguiente forma:

$$\text{review} = \text{review_scores_accuracy} \cdot 100\cdot \frac{\text{number_of_reviews}}{\sum \text{number_of_reviews}}$$

Esta sustituirá a las variables _number_of_reviews_ y _review_scores_accuracy_

In [ ]:
review = df['review_scores_accuracy']*(df['number_of_reviews']/sum(df['number_of_reviews'])*100)

# 2. División de los datos

In [ ]:
# Paquetes y liberias
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

En primer lugar, creamos la matriz de observaciones y el vector etiqueta con los que trabajaremos para ajustar el modelo de regresión lineal.

In [ ]:
# Vector etiqueta
y = df['price']

# Matriz de observaciones
X = df_log[['latitude', 'longitude','bathrooms', 'beds',  'security_deposit', 'cleaning_fee', 'extra_people',
            'Entire home/apt', 'Private room', 'Shared room']]
X.insert(7, 'review', review)

In [ ]:
# Estandarizamos los datos
objeto = StandardScaler()
X_ss = pd.DataFrame(objeto.fit_transform(X.iloc[:,0:8]), columns=list(X.iloc[:,0:8].columns))
# Concatenamos los datos
X = pd.concat([X_ss, X.loc[:,'Entire home/apt':'Shared room'].reset_index()], axis=1)
X.drop('index', axis=1, inplace=True)
X.head(5)

Separamos los datos en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Para finalizar, exportamos los datos.

In [ ]:
# Librerias
import os

# Carpeta con datos
if not os.path.exists('Datos'):
    os.makedirs('Datos')
    
# Guardamos los datos
X_train.to_csv('Datos/X_train.csv', index=False)
X_test.to_csv('Datos/X_test.csv', index=False)
y_train.to_csv('Datos/y_train.csv', index=False)
y_test.to_csv('Datos/y_test.csv', index=False)